# Kickoff

In [ ]:
thresh = 0.125 # threshold of initial predictions
num_thresh = 3 # number of prediction required to be used for final output
step = 30

nmin = 10
nmax = 2000 # Higher recall

IMSIZE = 1024
prediou = 0.55 # use qishen's value

fastcommit = True # if true, uses a shortened inference for commit.

# Detection models
check_end = "../input/nfleffdetmodel/effdet4-end-1024-epoch8.bin"
check_side = "../input/nfleffdetmodel/best-side-1024-effdet4-epoch6.bin"

# Classification models
checkpoint_classification_128_0 = "../input/classification-nfl/resnet3d_128_mixup_all_epoch6_fold0.pth"
checkpoint_classification_128_1 = "../input/classification-nfl/resnet3d_128_mixup_all_epoch7_fold0.pth"
checkpoint_classification_128_2 = "../input/classification-nfl/resnet3d_128_mixup_all_epoch8_fold0.pth"
checkpoint_classification_128_3 = "../input/classification-nfl/res3d-ishigamifold.pth"
checkpoint_classification_96_0 = "../input/nfl-classification-models/res3d_ishigamisplit_96x96_best.pth"
checkpoint_classification_96_1 = "../input/nfl-classification-models/res3d_96x96_ishigamisplit_last.pth"
checkpoint_classification_96_2 = "../input/nfl-classification-models/mc3_96x96_ishigamisplit_last.pth"
checkpoint_classification_96_3 = "../input/nfl-classification-models/mc3_96x96_ishigamisplit_best.pth"

In [ ]:
# SubmissionかCommit時かどうか存在するファイル名を見て確認する。commit時ならば高速化のために俵さんのデータセットを読みに行く。
import os
if os.path.exists("../input/nfl-impact-detection/test/57906_000718_Endzone.mp4"):
    commit = True
else:
    commit = False

In [ ]:
!pip install ../input/nfl-lib/timm-0.1.26-py3-none-any.whl
!tar xfz ../input/nfl-lib/pkgs.tgz
# for pytorch1.6
cmd = "sed -i -e 's/ \/ / \/\/ /' timm-efficientdet-pytorch/effdet/bench.py"
!$cmd

In [ ]:
import sys
sys.path.insert(0, "timm-efficientdet-pytorch")
sys.path.insert(0, "../input/ttach-kaggle/ttach")
sys.path.insert(0, "omegaconf")
sys.path.insert(0, "../input/odachkaggle/ODA-Object-Detection-ttA-main")
import odach as oda
import ttach as tta

import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
import pandas as pd
import gc
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchEval
from effdet.efficientdet import HeadNet
import warnings

warnings.filterwarnings("ignore")
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(SEED)

In [ ]:
def mk_images(video_name, video_labels, video_dir, out_dir, only_with_impact=True):
    video_path=f"{video_dir}/{video_name}"
    video_name = os.path.basename(video_path)
    vidcap = cv2.VideoCapture(video_path)
    if only_with_impact:
        boxes_all = video_labels.query("video == @video_name")
        print(video_path, boxes_all[boxes_all.impact == 1.0].shape[0])
    else:
        print(video_path)
    frame = 0
    while True:
        it_worked, img = vidcap.read()
        if not it_worked:
            break
        frame += 1
        if only_with_impact:
            boxes = video_labels.query("video == @video_name and frame == @frame")
            boxes_with_impact = boxes[boxes.impact == 1.0]
            if boxes_with_impact.shape[0] == 0:
                continue
        img_name = f"{video_name}_frame{frame}"
        image_path = f'{out_dir}/{video_name}'.replace('.mp4',f'_{str(frame).zfill(3)}.png')
        _ = cv2.imwrite(image_path, img)

In [ ]:
DATA_ROOT_PATH = 'test_images'
# Use dataset if in commitmode to save GPU
if commit:
    DATA_ROOT_PATH= "../input/nfl-impact-detection-train-frames"
out_dir = DATA_ROOT_PATH

video_dir = '/kaggle/input/nfl-impact-detection/test'
uniq_video = sorted([path.split('/')[-1] for path in glob(f'{video_dir}/*.mp4')])
print(uniq_video)

# Generate images if in Test mode
if not os.path.exists(out_dir):
    !mkdir -p $out_dir
    for video_name in uniq_video:
        mk_images(video_name, pd.DataFrame(), video_dir, out_dir, only_with_impact=False)

In [ ]:
def get_valid_transforms():
    return A.Compose([
            A.Resize(height=IMSIZE, width=IMSIZE, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [ ]:
class DatasetRetriever(Dataset):
    def __init__(self, image_ids, dir=None, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms
        self.dir = dir

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        if not commit:
            image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}', cv2.IMREAD_COLOR).copy().astype(np.float32)
        else:
            image = cv2.imread(f'{DATA_ROOT_PATH}/{self.dir}/{image_id}', cv2.IMREAD_COLOR).copy().astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

## make classification models

In [ ]:
import torchvision


def load_classification_model(ckpt_path: str):
    if "mc3" in ckpt_path:
        model = torchvision.models.video.mc3_18(pretrained=False)
    else:
        model = torchvision.models.video.r3d_18(pretrained=False)
    model.fc = torch.nn.Linear(512, 1)
    
    ckpt = torch.load(ckpt_path)
    if "model_state_dict" in ckpt.keys():
        model.load_state_dict(ckpt["model_state_dict"])
    else:
        model.load_state_dict(ckpt)
    model.to("cuda").eval()
    return model

In [ ]:
model_128_0 = load_classification_model(checkpoint_classification_128_0)
model_128_1 = load_classification_model(checkpoint_classification_128_1)
model_128_2 = load_classification_model(checkpoint_classification_128_2)
model_128_3 = load_classification_model(checkpoint_classification_128_3)
model_96_0 = load_classification_model(checkpoint_classification_96_0)
model_96_1 = load_classification_model(checkpoint_classification_96_1)
model_96_2 = load_classification_model(checkpoint_classification_96_2)
model_96_3 = load_classification_model(checkpoint_classification_96_3)

models_128 = [model_128_0, model_128_1, model_128_2, model_128_3]
models_96 = []

!nvidia-smi

## detector misc

In [ ]:
def load_net(checkpoint_path):
    if "effdet4"in checkpoint_path:
        config = get_efficientdet_config('tf_efficientdet_d4')
    else:
        config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)
    config.num_classes = 2
    config.image_size=IMSIZE
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))
    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])
    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

netend = load_net(check_end)
netside = load_net(check_side)

In [ ]:
def bb_intersection_over_union(A, B) -> float:
    xA = max(A[0], B[0])
    yA = max(A[1], B[1])
    xB = min(A[2], B[2])
    yB = min(A[3], B[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)

    if interArea == 0:
        return 0.0

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (A[2] - A[0]) * (A[3] - A[1])
    boxBArea = (B[2] - B[0]) * (B[3] - B[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def df2box(df):
    return np.array([df["x"],df["y"],df["w"]+df["x"],df["h"]+df["y"]]).T

In [ ]:
def make_predictions(images, net, score_threshold=0.1):
    images = torch.stack(images).cuda().float()
    predictions = []
    with torch.no_grad():
        det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
        for i in range(images.shape[0]):
            boxes = det[i].detach().cpu().numpy()[:,:4]    
            scores = det[i].detach().cpu().numpy()[:,4]
            labels = det[i].detach().cpu().numpy()[:,5]
            indexes = np.where((scores > score_threshold)*(labels==2))[0]
            #boxes = boxes[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            predictions.append({
                'boxes': boxes[indexes],
                'scores': scores[indexes],
                'labels': labels[indexes],
            })
    return [predictions]

from odach.wbf import *
def run_wbf(predictions, image_index, image_size=IMSIZE, iou_thr=prediou, skip_box_thr=thresh, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    
    labels = [prediction[image_index]['labels'].tolist() for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

import matplotlib.pyplot as plt

## filter functions..

In [ ]:
def find_match(boxes_list, new_box, idx, target_iou):
    best_index = []
    for i in range(len(boxes_list)):
        box = boxes_list[i]

        iou = bb_intersection_over_union(box, new_box)
        if iou > target_iou:
            best_index.append(idx[i])

    return best_index

def find_matching_box(boxes_list, new_box, idx, target_iou):
    best_index = []
    for i in range(len(boxes_list)):
        box = boxes_list[i]

        iou = bb_intersection_over_union(box, new_box)
        if iou > target_iou:
            best_index.append(idx[i])

    return best_index

# 1st stage: Detection

In [ ]:
def predict(imgs, net, dir=None):
    
    dataset = DatasetRetriever(
    image_ids=imgs,
    dir=dir,
    transforms=get_valid_transforms()
    )

    def collate_fn(batch):
        return tuple(zip(*batch))

    data_loader = DataLoader(
        dataset,
        batch_size=8,
        shuffle=False,
        num_workers=4,
        drop_last=False,
        collate_fn=collate_fn
    )

    result_image_ids = []
    results_boxes = []
    results_scores = []
    results_frames = []
    cnt = 0
    # Inference!
    for images, image_ids in data_loader:
        predictions = make_predictions(images, net)
        for i, image in enumerate(images):
            box_list, score_list, label_list = run_wbf(predictions, image_index=i, skip_box_thr=thresh)
            boxes = box_list
            scores = score_list
            image_id = image_ids[i]
            boxes[:, 0] = (boxes[:, 0] * 1280 / IMSIZE)
            boxes[:, 1] = (boxes[:, 1] * 720 / IMSIZE)
            boxes[:, 2] = (boxes[:, 2] * 1280 / IMSIZE)
            boxes[:, 3] = (boxes[:, 3] * 720 / IMSIZE)
            boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
            boxes = boxes.astype(np.int32)
            boxes[:, 0] = boxes[:, 0].clip(min=0, max=1280-1)
            boxes[:, 2] = boxes[:, 2].clip(min=0, max=1280-1)
            boxes[:, 1] = boxes[:, 1].clip(min=0, max=720-1)
            boxes[:, 3] = boxes[:, 3].clip(min=0, max=720-1)
            result_image_ids += [image_id]*len(boxes)
            results_boxes.append(boxes)
            results_scores.append(scores)
            cnt+=1
    
    box_df = pd.DataFrame(np.concatenate(results_boxes), columns=['x', 'y', 'w', 'h'])
    test_df = pd.DataFrame({'scores':np.concatenate(results_scores), 'image_name':result_image_ids})
    test_df = pd.concat([test_df, box_df], axis=1)
    test_df['frame'] = test_df.image_name.str.split('_').str[3].str.replace('.png','').astype(int)
    test_df['gameKey'] = test_df.image_name.str.split('_').str[0].astype(int)
    test_df['playID'] = test_df.image_name.str.split('_').str[1].astype(int)
    test_df['view'] = test_df.image_name.str.split('_').str[2]
    test_df['frame'] = test_df.image_name.str.split('_').str[3].str.replace('.png','').astype(int)
    test_df['video'] = test_df.image_name.str.rsplit('_',1).str[0] + '.mp4'
    test_df.sort_values('frame', inplace=True)
    return test_df

In [ ]:
if fastcommit and commit:
    uniq_video = uniq_video[0:2]
    
print(uniq_video)

for iii,videoname in enumerate(uniq_video[::2]):
    videoend = videoname
    videoside = videoname[:-11]+"Sideline.mp4"
    print("videoend:{}, videoside:{}".format(videoend, videoside))
    
    ######################################
    # Inference Videos
    ######################################
    for phase, vid in enumerate([videoend, videoside]):
        # clear
        if commit:
            print(f'{DATA_ROOT_PATH}/{vid[:-4]}/')
            imgs = np.array([path.split('/')[-1] for path in glob(f'{DATA_ROOT_PATH}/{vid[:-4]}/*.png')])
        else:
            # for testmode
            imgs = np.array([path.split('/')[-1] for path in glob(f'{DATA_ROOT_PATH}/{vid[:-4]}*.png')])
            
        # Get predictions
        if phase == 0:
            end_df = predict(imgs, netend, dir=vid[:-4])
            print("end len:", len(end_df))
        else:
            side_df = predict(imgs, netside, dir=vid[:-4])
            print("side len:", len(side_df))
    
    final_side = side_df[['x', 'y', 'w', 'h', 'frame']].values
    final_end = end_df[['x', 'y', 'w', 'h', 'frame']].values
    
    # Make submission for both Side and Ends
    for phase, final_outs in enumerate([final_end, final_side]):
        if phase == 0:
            result_image_ids = []
            for i in final_outs:
                result_image_ids.append(end_df.loc[0].image_name)
        else:
            result_image_ids = []
            for i in final_outs:
                result_image_ids.append(side_df.loc[0].image_name)
        print(final_outs.shape)   

        # make df if final_out has boxes..
        try:
            box_df = pd.DataFrame(final_outs[:, 0:4].astype(int), columns=['left', 'top', 'width', 'height'])
            test_df = pd.DataFrame({'image_name':result_image_ids})
            test_df = pd.concat([test_df, box_df], axis=1)
            test_df['frame'] = final_outs[:,4].astype(int)
            print(test_df.head())

            #gameKey,playID,view,video,frame,left,width,top,height
            test_df['gameKey'] = test_df.image_name.str.split('_').str[0].astype(int)
            test_df['playID'] = test_df.image_name.str.split('_').str[1].astype(int)
            test_df['view'] = test_df.image_name.str.split('_').str[2]
            test_df['video'] = test_df.image_name.str.rsplit('_',1).str[0] + '.mp4'
            test_df = test_df[["gameKey","playID","view","video","frame","left","width","top","height"]]
            # concat
            if (iii==0) and (phase==0):
                submit_df=test_df
            else:
                submit_df=pd.concat([test_df, submit_df], axis=0)
        except:
            # no prediction found
            print("boxes not found")

In [ ]:
submit_df

# 2nd Stage: Classification

In [ ]:
def filter_ops(positive_df):
    # New Post-Processing Function!
    start = positive_df["frame"].min()
    end = positive_df["frame"].max()
    step = 30
    target_iou = 0.25

    boxes = positive_df[['x', 'y', 'w', 'h', 'frame']].values
    boxes[:,2] = boxes[:,0] + boxes[:,2]
    boxes[:,3] = boxes[:,1] + boxes[:,3]
    keep = np.ones(len(boxes))
    idx = np.array(range(len(boxes)))

    outs = []
    
    # Time-nms
    for t in np.arange(start, end, step):
        keys = (boxes[:,4]>=t)*((boxes[:,4]<=t+step)*(keep==1))
        loop_boxes_init = boxes[keys]
        loop_boxes = loop_boxes_init
        loop_idx = idx[keys]

        for box in loop_boxes_init:
            # Find box that overlaps with box
            hits = find_matching_box(loop_boxes, box, loop_idx, target_iou)
            keep[hits] = 0
            outs.append(hits)
            # Remove the hit boxes from loop_boxes
            keys = (boxes[:,4]>=t)*((boxes[:,4]<=t+step)*(keep==1))
            loop_boxes = boxes[keys]
            loop_idx = idx[keys]

    # filter outputs by nms
    choose_centerframe = False
    final_outs = []
    for out in outs:
        if len(out)>=num_thresh: # threshold of detection num!
            box = boxes[out]
            o = np.array(np.median(box, axis=0))
            # TODO: 最もスコアが高いものを選択
            final_outs.append(o)
    return np.array(final_outs)

In [ ]:
def threshold_opt(preds_df, thresh, nmax=25, nmin=15):
    videos = preds_df.video.unique()
    for i,video in enumerate(videos):
        dfs = preds_df[preds_df["video"]==video].reset_index()
        # Inference
        val_dataset = HeadClassificationDataset(dfs, image_dir=Path(""), img_size=128, transforms=get_valid_transforms(128), transforms2=get_valid_transforms(96))
        val_loader = torchdata.DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=4)
        targets = []
        count = 0
        for img_128, img_96 in val_loader:
            img_128 = img_128.to("cuda")
            img_96 = img_96.to("cuda")
            tmp = []
            for model in models_128:
                target = (torch.sigmoid(model(img_128)) + torch.sigmoid(model(torch.flip(img_128, [4])))) / 2
                tmp.append(target.cpu().detach().numpy())

            for model in models_96:
                target = (torch.sigmoid(model(img_96)) + torch.sigmoid(model(torch.flip(img_96, [4])))) / 2
                tmp.append(target.cpu().detach().numpy())

            targets.append(np.mean(tmp, axis=0))
            count += 1
        # to array
        ts = []
        for t in targets:
            ts.append(t[0])
        targets = np.array(ts).reshape(-1)

        # threshold loop
        run_th = thresh
        numbox = 0
        tries = 0
        while (numbox>nmax or numbox<nmin) and tries<=20:
            try:
                positive_df = dfs[targets>run_th]
                pred = positive_df[positive_df["video"]==video]
                final_outs = filter_ops(pred)
                numbox = len(final_outs)
            except:
                pass
            if numbox>nmax:
                run_th += 0.02
            elif numbox<nmin:
                run_th -= 0.02
            tries += 1
            
        final_outs[:,2] = final_outs[:,2] - final_outs[:,0]
        final_outs[:,3] = final_outs[:,3] - final_outs[:,1]
        # to dataframe
        box_df = pd.DataFrame(final_outs[:, 0:4].astype(int), columns=['left', 'top', 'width', 'height'])
        test_df = pd.DataFrame({'gameKey':dfs.gameKey[:len(box_df)], "playID":dfs.playID[:len(box_df)], "view":dfs.view[:len(box_df)], "video":dfs.video[:len(box_df)]})
        test_df = pd.concat([test_df, box_df], axis=1)
        test_df['frame'] = final_outs[:,4].astype(int)

        #gameKey,playID,view,video,frame,left,width,top,height
        test_df = test_df[["gameKey","playID","view","video","frame","left","width","top","height"]]

        if i == 0:
            out_df = test_df
        else:
            out_df = pd.concat([out_df, test_df], axis=0)
    return out_df

In [ ]:
import torch.utils.data as torchdata
from pathlib import Path
class HeadClassificationDataset(torchdata.Dataset):
    def __init__(self, df: pd.DataFrame, image_dir: Path, img_size=128, transforms=None, transforms2=None):
        self.df = df
        self.image_dir = image_dir
        self.transforms = transforms
        self.transforms2 = transforms2
        self.img_size = img_size
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx: int):
        sample = self.df.loc[idx, :]
        x, y, w, h,frame = sample.x, sample.y, sample.w, sample.h, sample.frame
        #x = x + (np.random.randn(1)).astype(int)
        #y = y + (np.random.randn(1)).astype(int)
        #w = w + (np.random.randn(1)*2).astype(int)
        #h = h + (np.random.randn(1)*2).astype(int)
        image_id = sample.video[:-4]
        vid = sample.video
        frame_idx = sample.frame
        prefix = image_id
        if commit:
            image_dir = Path(f"{DATA_ROOT_PATH}/{vid[:-4]}/")
        else:
            # for testmode
            image_dir = Path(f"{DATA_ROOT_PATH}")
        try:
            all_images = []
            for frame_diff in [-4, -3, -2, -1, 0, 1, 2, 3, 4]:
                image_id = prefix + '_' + str(frame_idx+frame_diff).zfill(3) + '.png'
                image = cv2.imread(str(image_dir / image_id))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0))
                all_images.append(image)
        except:
            print("oops")
            all_images = []
            for frame_diff in [-0, -0, -0, -0, 0, 0, 0, 0, 0]:                    
                image_id = prefix + '_' + str(frame_idx+frame_diff).zfill(3) + '.png'
                print(image_id)
                image = cv2.imread(str(image_dir / image_id))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0))
                all_images.append(image)
        x, y, w, h,frame = sample.x, sample.y, sample.w, sample.h, sample.frame
        all_images = np.concatenate(all_images, axis=2)
        woffset = (self.img_size - w) // 2
        hoffset = (self.img_size - h) // 2
        left = max(x - woffset, 0)
        right = min(left + self.img_size, all_images.shape[1])
        top = max(y - hoffset, 0)
        bottom = min(top + self.img_size, all_images.shape[0])
        #print(top)
        #print(all_images.shape)
        cropped = all_images[top:bottom, left:right, :].astype(np.float32)
        cropped /= 255.0
        if self.transforms is not None:
            cropped = self.transforms(image=cropped)["image"]
        cropped = cropped.view(9,3,self.img_size,self.img_size).transpose(1,0)
        ### 96 IMAGES
        woffset = (96 - w) // 2
        hoffset = (96 - h) // 2
        left = max(x - woffset, 0)
        right = min(left + 96, all_images.shape[1])
        top = max(y - hoffset, 0)
        bottom = min(top + 96, all_images.shape[0])
        #print(top)
        #print(all_images.shape)
        cropped2 = all_images[top:bottom, left:right, :].astype(np.float32)
        cropped2 /= 255.0
        if self.transforms2 is not None:
            cropped2 = self.transforms2(image=cropped2)["image"]
        cropped2 = cropped2.view(9, 3, 96, 96).transpose(1,0)
        return cropped, cropped2

def get_valid_transforms(img_size=128):
    return A.Compose([
        A.Resize(height=img_size, width=img_size, p=1),
        ToTensorV2(p=1.0)
    ], p=1.0)

In [ ]:
submit_df["x"] = submit_df["left"]
submit_df["y"] = submit_df["top"]
submit_df["w"] = submit_df["width"]
submit_df["h"] = submit_df["height"]

submit_df = submit_df[submit_df["frame"]>=10]

In [ ]:
submit_df2 = threshold_opt(submit_df, 0.9, nmax=25, nmin=15)
print(len(submit_df), len(submit_df2))
submit_df2.reset_index(inplace=True, drop=True)
submit_df2.head()

In [ ]:
submit_df2.head()

In [ ]:
# clearing working dir
# be careful when running this code on local environment!
# !rm -rf *
!mv * /tmp/

In [ ]:
import nflimpact
env = nflimpact.make_env()
env.predict(submit_df2) # df is a pandas dataframe of your entire submission file